In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import os
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# internal imports
from dynamics import mld
from warm_start_hmpc.controller import HybridModelPredictiveController

In [ ]:
# controller horizon
T = 25

# weight matrices
C = np.eye(mld.nx) * 1.
C_T = np.eye(mld.nx) * 1.
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T * 1.
objective = [C, D, C_T]

# terminal constraints
F_T = np.vstack((np.eye(mld.nx), - np.eye(mld.nx)))
h_T = np.ones(2*mld.nx)*.5
terminal_set = [F_T, h_T]

# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

In [ ]:
x0 = np.array([0., 0., 1., 0.])

In [ ]:
# solve in open loop
uc, ub, x, objective, leaves = controller.feedforward(x0, draw_label='Cart pole with walls')

In [ ]:
e0 = np.zeros(mld.nx)
controller.construct_warm_start(leaves,x0, uc[0], ub[0], e0)

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(incumbent.additional.primal.x)

In [ ]:
controller._get_bounds_on_binaries